
#  Module 9: Web-Based Data Visualization (D3 & Plotly)
**Duration:** 9 Hours (3 Sessions × 3 Hours)  
**Dataset:** Titanic Dataset  
**Goal:** Learn to build interactive, web-based visualizations using **D3.js** and **Plotly Dash**.



##  Learning Objectives
By the end of this module, students will:
- Understand D3.js core principles (DOM manipulation, method chaining, data binding)
- Build interactive SVG-based visualizations with D3
- Create interactive web dashboards using Plotly and Plotly Dash
- Visualize Titanic survival patterns interactively
- Deploy a web-based dashboard locally

#  **Session 2: Plotly Express Interactive Visualizations**
We'll now use **Plotly Express** in Python to visualize Titanic survival data.

In [5]:
# First, install and import necessary libraries
!pip install plotly pandas

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Load Titanic dataset
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Clean the data
df = df.dropna(subset=['Age', 'Fare'])
print(f"Dataset loaded with {len(df)} records")
print(df[['Age', 'Fare', 'Survived', 'Pclass']].head())

Dataset loaded with 714 records
    Age     Fare  Survived  Pclass
0  22.0   7.2500         0       3
1  38.0  71.2833         1       1
2  26.0   7.9250         1       3
3  35.0  53.1000         1       1
4  35.0   8.0500         0       3


In [6]:
# Create an interactive scatter plot (similar to D3)
fig = px.scatter(
    df,
    x='Age',
    y='Fare',
    color='Survived',
    color_discrete_map={'0': 'red', '1': 'blue'},
    title='Titanic: Age vs Fare (D3 Style)',
    labels={'Age': 'Age', 'Fare': 'Fare'},
    hover_data=['Pclass', 'Sex', 'Name'],
    opacity=0.7
)

# Customize to look more like D3
fig.update_traces(marker=dict(size=6))
fig.update_layout(
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='lightgray')
)

fig.show()

In [7]:
# Survival by Class - Bar Chart
survival_by_class = df.groupby(['Pclass', 'Survived']).size().reset_index(name='count')
survival_by_class['Survived'] = survival_by_class['Survived'].map({0: 'Died', 1: 'Survived'})

fig2 = px.bar(
    survival_by_class,
    x='Pclass',
    y='count',
    color='Survived',
    barmode='group',
    title='Survival by Passenger Class',
    labels={'Pclass': 'Passenger Class', 'count': 'Number of Passengers'}
)

fig2.show()

In [9]:
from IPython.display import HTML, IFrame
import json

# Create HTML with embedded D3.js
d3_html = """
<!DOCTYPE html>
<html>
<head>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <style>
        .circle { opacity: 0.7; }
        .axis { font-size: 12px; }
    </style>
</head>
<body>
    <div id="chart"></div>
    <script>
        // Titanic data embedded directly
        const data = """ + df[['Age', 'Fare', 'Survived']].dropna().to_json(orient='records') + """;

        const width = 600, height = 400;
        const margin = { top: 20, right: 30, bottom: 40, left: 50 };

        const svg = d3.select("#chart")
            .append("svg")
            .attr("width", width)
            .attr("height", height);

        const xScale = d3.scaleLinear()
            .domain([0, d3.max(data, d => d.Age)])
            .range([margin.left, width - margin.right]);

        const yScale = d3.scaleLinear()
            .domain([0, d3.max(data, d => d.Fare)])
            .range([height - margin.bottom, margin.top]);

        // Add circles
        svg.selectAll("circle")
            .data(data)
            .enter()
            .append("circle")
            .attr("class", "circle")
            .attr("cx", d => xScale(d.Age))
            .attr("cy", d => yScale(d.Fare))
            .attr("r", 4)
            .attr("fill", d => d.Survived === 1 ? "blue" : "red");

        // Add axes
        const xAxis = d3.axisBottom(xScale);
        const yAxis = d3.axisLeft(yScale);

        svg.append("g")
            .attr("class", "axis")
            .attr("transform", `translate(0,${height - margin.bottom})`)
            .call(xAxis);

        svg.append("g")
            .attr("class", "axis")
            .attr("transform", `translate(${margin.left},0)`)
            .call(yAxis);

        // Add labels
        svg.append("text")
            .attr("x", width / 2)
            .attr("y", height - 5)
            .attr("text-anchor", "middle")
            .text("Age");

        svg.append("text")
            .attr("transform", "rotate(-90)")
            .attr("x", -height / 2)
            .attr("y", 15)
            .attr("text-anchor", "middle")
            .text("Fare");
    </script>
</body>
</html>
"""

# Display the D3 visualization
HTML(d3_html)